# 高效微调
总结：
1. XXConfig(task_type=XXX)
2. get_peft_model(model, config)
3. PeftModel.from pretrained(model, model id)
4. peft model.merge and unload()

## 1.BitFit

BitFit（Bias-term Fine-tuning）是一种参数高效的微调方法，专注于仅调整预训练模型中的偏置项（bias），而冻结其他参数。这种方法在中小规模数据集上表现出色，能够在仅更新约0.08%的参数情况下，达到与全量微调相当的效果。 

主要特点：

参数更新量小： 仅调整模型的偏置项，显著减少了需要更新的参数数量。

适用性广： 在中小规模数据集上，BitFit的性能与全量微调相当，甚至在某些任务上表现更佳。

资源友好： 由于更新的参数量极少，适合在内存受限的环境中部署，为硬件部署提供了可能性。

应用场景： BitFit适用于需要在有限资源下进行模型微调的场景，特别是在中小规模数据集上，能够在保持性能的同时，降低计算和存储成本。

限制场景： 对偏置项敏感的任务（如分类任务）表现较好，但对于需要调整大量模型参数的复杂任务（如生成任务、问答系统）可能效果有限。
偏置项对模型功能的贡献有限，不能充分捕获任务相关的复杂特征

In [ ]:
# 冻结所有参数
for param in model.parameters():
    param.requires_grad = False

# 只解冻偏置项
for name, param in model.named_parameters():
    if "bias" in name:
        param.requires_grad = True


## P-Tuning

可以理解为给模型的 embed_tokens 层扩展出 N 个（或者指定数量的）虚拟 token 对应的嵌入向量，然后将这些虚拟 token 的嵌入拼接到输入序列前，作为模型的完整输入。

优点

参数高效：只优化少量 Prompt 嵌入 (<1%)，节省显存和计算资源。

适配灵活：冻结模型主干，适配多个任务无需修改原始模型。

训练效率高：只训练 Prompt 嵌入，训练速度快，开销小。

适合生成任务：在文本生成任务中（如翻译、摘要）效果尤为显著。

缺点

表达能力有限：对复杂任务（如分布迁移）效果不如 LoRA 或全量微调。

依赖预训练模型：性能高度依赖模型的原始表示能力。

Prompt 长度需调优：长度过短效果不佳，过长可能引入噪声。

泛化能力受限：少量参数可能导致过拟合。

应用场景

低资源适配：显存或计算受限时对大模型进行任务微调。

多任务支持：为每个任务训练独立 Prompt 嵌入。

Few-Shot/Zero-Shot 学习：少量或无标注数据的任务。

领域适配：特定领域（如医疗、法律）的迁移学习。

生成任务：翻译、摘要、格式化输出。

In [ ]:
# PEFT 实现
from peft import PromptTuningConfig, get_peft_model, TaskType
# 配置 Prompt Tuning
config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,  # SEQ_CLS SEQ_2_SEQ_LM  CAUSAL_LM
    prompt_length=10, # 一般从 10-50 的范围开始实验，根据验证集效果调整。
    num_virtual_tokens=10,  
)

peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()

In [ ]:
# 非PEFT实现
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 冻结所有参数
for param in model.parameters():
    param.requires_grad = False

# 添加可学习的 Prompt 嵌入
prompt_length = 5  
hidden_size = model.config.hidden_size
prompt_embeddings = torch.nn.Embedding(prompt_length, hidden_size)
torch.nn.init.normal_(prompt_embeddings.weight, mean=0.0, std=0.02)

# 定义训练输入
inputs = tokenizer("The movie was great!", return_tensors="pt")
input_ids = inputs["input_ids"]

# 拼接可学习 Prompt
prompt_ids = torch.arange(prompt_length).unsqueeze(0)
prompt_vectors = prompt_embeddings(prompt_ids)
prompt_vectors = prompt_vectors.repeat(input_ids.size(0), 1, 1)
inputs_embeds = model.base_model.embeddings(input_ids)
inputs_embeds = torch.cat([prompt_vectors, inputs_embeds], dim=1)


outputs = model(inputs_embeds=inputs_embeds, labels=torch.tensor([1]))
loss = outputs.loss
print("Loss:", loss.item())


上面的是soft prompt，还有hard prompt，即以真实的文本形式出现，如 "Classify the sentiment of this sentence: [INPUT]"，这种方式不训练。

## P-Tuning v2

P-Tuning v2 是一种改进的 Prompt Tuning 技术，主要解决了原始 Prompt Tuning 方法在小规模模型和复杂任务上的局限性，同时在大模型上进一步提升了性能。

P-Tuning v2 的核心改进

支持任意规模模型：

原始 Prompt Tuning 的性能依赖于大模型（如 GPT-3），而 P-Tuning v2 通过优化结构和训练流程，使得小模型（如 BERT）也能从 Prompt Tuning 中受益。
优化的提示嵌入插入位置：

不局限于将 Prompt 嵌入放在输入序列的开头，而是可以动态插入到模型的多个层中（如 Transformer 的中间层）。
这种分层插入（Layer-Wise Prompt Injection）机制提升了 Prompt 的表达能力。
结合深层 Transformer 表示：

Prompt 不再仅通过输入引导模型，而是直接参与深层 Transformer 的内部信息表示。
通过与不同层的信息交互，Prompt 嵌入能够捕获更复杂的任务相关特征。
优化训练策略：

通过更高效的初始化和优化策略，使 Prompt 嵌入更快收敛，同时提高泛化性能。

In [ ]:
# PEFT实现
p_tuning_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,  
    prompt_length=20,            
    num_virtual_tokens=20,       
    encoder_hidden_states=False, 
    insert_into_layers="deep",   
)

In [ ]:
p_tuning_config = PromptTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  
    prompt_length=30,                 
    num_virtual_tokens=30,            
    encoder_hidden_states=False,       
    insert_into_layers="custom"       
)


p_tuning_config.layers_to_insert = [0, 3, 7, 10]  
p_tuning_config.prompt_location = "start"    # strart middle end，middle需要自定义中间插入逻辑
p_tuning_config.shared_prompt = True    # False适合复杂任务         


浅插入（shallow）：提示嵌入只在输入层使用，后续层不再加入提示嵌入。

深插入（deep）：提示嵌入被动态插入到多个 Transformer 中间层，增强提示信息的影响力。

encoder_hidden_states：如果任务需要处理多模态数据（如图像和文本），图像特征或文本特征可以作为额外隐藏状态，增强提示嵌入的表达能力。

示例：
图像描述生成任务中，图像的编码器隐藏状态可以传递给提示嵌入，指导文本生成。

如果任务简单且资源受限，推荐设置为 False；如果任务复杂且需要丰富的上下文信息，建议启用 True

自回归模型不分编码器和解码器，encoder_hidden_states 无意义。

## Prefix-Tuning

和P-Tuning对比，参考https://blog.csdn.net/weixin_43863869/article/details/134760405

Prefix-Tuning 的训练部分是前缀嵌入（Prefix Embedding），即加入到 Transformer 每一层自注意力机制中的一组可学习参数。
这部分参数是独立于模型主体的，只有前缀嵌入会被优化。

Prefix-Tuning 更适合生成任务和复杂任务，而 P-Tuning 更适合分类任务和简单场景。根据任务需求选择合适的方法即可！

In [ ]:
# PEFT实现
from peft import PrefixTuningConfig, get_peft_model, TaskType

prefix_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  
    num_virtual_tokens=20,            
    encoder_hidden_states=False        
)

peft_model = get_peft_model(model, prefix_config)

## Lora

In [ ]:
# PEFT实现，非PEFT实现不推荐写，会有可训练部分的问题要处理，这个轮子比较好用
from peft import LoraConfig, get_peft_model, PeftModel
for name, param in model.named_parameters():
    param.requires_grad = True  
    lora_config = LoraConfig(
        r=4,
        lora_alpha=8,
        target_modules=["q_proj"],
        lora_dropout=0.05,
        bias="none",
    )
    model = get_peft_model(model, lora_config)
    for name, param in model.model.named_parameters():
        if 'lora' not in name:
            param.requires_grad = False
            
# 加载
lora_model = PeftModel.from_pretrained(base_model, lora_save_path)
#合并保存
merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained(merged_model_save_path)

In [ ]:
#分流，禁用
import torch.nn as nn
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2, config1)
model2.save_pretrained("./loraA")

config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="taskA")
model2.load_adapter("./loraB/", adapter_name="taskB")

model2.active_adapter # 此时是taskA
model2.set_adapter("taskB")
with model2.disable_adapter():
    print(model2(torch.arange(0, 10).view(1, 10).float()))

## IA3

参考https://blog.csdn.net/LLMUZI123456789/article/details/136880839

IA3 (Infused Adapter by Attention and Activation) 是一种高效的参数微调方法，专为大规模预训练模型（如 Transformers）设计。IA3 的核心思想是：通过引入 可学习的标量缩放因子，调整每一层中 注意力权重（Attention） 和 前馈激活值（Activation） 的幅度，而不修改模型本身的参数。

缩放 Attention 和 FFN，适合注意力主导的任务（如分类、生成）

In [ ]:
# PEFT实现
from peft import IA3Config, get_peft_model, TaskType, PeftModel
ia3_config = IA3Config(
    task_type=TaskType.SEQ_2_SEQ_LM,  
    target_modules=["v_proj", "ffn"], 
    ia3_alpha=8,                      
    bias="none",                      
)

ia3_model = get_peft_model(model, ia3_config)
ia3_model.print_trainable_parameters()
ia3_model = PeftModel.from_pretrained(base_model, "./ia3_model")

默认设置：ia3_alpha=8(÷8)

对任务敏感时：减小 ia3_alpha，例如设置为 4，让模型更灵活地调整注意力或激活值。

对任务要求较稳健时：增大 ia3_alpha，例如设置为 16，避免过大改动。

默认设置：bias="none"

none or all or lora_only

all-对所有 Transformer 模块的偏置项进行调整。

lora_only-偏置项仅在与 IA3 相关的模块（如 Value 投影矩阵）中进行调整。

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from peft import PeftModel, PeftConfig
import json

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "/home/zhuyao/Sunpeng/models/qwen_2B_instruct", torch_dtype="auto", device_map="cuda"
)

In [5]:
model.model

Qwen2VLModel(
  (embed_tokens): Embedding(151936, 1536)
  (layers): ModuleList(
    (0-27): 28 x Qwen2VLDecoderLayer(
      (self_attn): Qwen2VLSdpaAttention(
        (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
        (k_proj): Linear(in_features=1536, out_features=256, bias=True)
        (v_proj): Linear(in_features=1536, out_features=256, bias=True)
        (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        (rotary_emb): Qwen2VLRotaryEmbedding()
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
        (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
        (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((1536,), eps=1e-06)
  (rotary_emb): Qwen2VLRotaryEmb